In [1]:
# pytorch
import torch
import torchvision
from torchvision import transforms, datasets, models
from torchsummary import summary
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# other
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import glob
from tqdm import tqdm

In [2]:
class appleDataset(Dataset):
    def __init__(self, path, train, transform=None):
        self.path = path
        if train:
            self.AbNorm_path = path + '/train/AbNorm/'
            self.Mot_path = path + '/train/Mot/'
            self.Norm_path = path + '/train/Norm/'
        else:
            self.AbNorm_path = path + '/test/AbNorm/'
            self.Mot_path = path + '/test/Mot/'
            self.Norm_path = path + '/test/Norm/'
        
        self.AbNorm_img_list = glob.glob(self.AbNorm_path + '/*.jpg')
        self.Mot_img_list = glob.glob(self.Mot_path + '/*.jpg')
        self.Norm_img_list = glob.glob(self.Norm_path + '/*.jpg')

        self.transform = transform

        self.img_list = self.AbNorm_img_list + self.Mot_img_list + self.Norm_img_list
        self.class_list = [0] * len(self.AbNorm_img_list) + [1] * len(self.Mot_img_list) + [2] * len(self.Norm_img_list)
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = self.img_list[idx]
        label = self.class_list[idx]
        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)

        return img, label


In [3]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = appleDataset(path='./input', train=True, transform=transform)
trainloader = DataLoader(dataset=train_dataset,
                        batch_size=512,
                        shuffle=True,
                        drop_last=False)
    
test_dataset = appleDataset(path='./input', train=False, transform=transform)
testloader = DataLoader(dataset=test_dataset,
                        batch_size=256,
                        shuffle=True,
                        drop_last=False
                        )

In [4]:
classes = ('Norm','Mot','AbNorm')

In [5]:
resnet_pt = models.resnet18(weights=True)

C:\Users\2469l\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
unfreeze = ['layer4.0.conv1.weight', 'layer4.0.bn1.weight', 'layer4.0.bn1.bias', 'layer4.0.bn1.running_mean', 'layer4.0.bn1.running_var', 'layer4.0.bn1.num_batches_tracked', 'layer4.0.conv2.weight', 'layer4.0.bn2.weight', 'layer4.0.bn2.bias', 'layer4.0.bn2.running_mean', 'layer4.0.bn2.running_var', 'layer4.0.bn2.num_batches_tracked', 'layer4.0.downsample.0.weight', 'layer4.0.downsample.1.weight', 'layer4.0.downsample.1.bias', 'layer4.0.downsample.1.running_mean', 'layer4.0.downsample.1.running_var', 'layer4.0.downsample.1.num_batches_tracked', 'layer4.1.conv1.weight', 'layer4.1.bn1.weight', 'layer4.1.bn1.bias', 'layer4.1.bn1.running_mean', 'layer4.1.bn1.running_var', 'layer4.1.bn1.num_batches_tracked', 'layer4.1.conv2.weight', 'layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'layer4.1.bn2.running_mean', 'layer4.1.bn2.running_var', 'layer4.1.bn2.num_batches_tracked', 'fc.weight', 'fc.bias'] 

In [7]:
resnet_pt = models.resnet18(weights=True)
# freezing
for name, param in resnet_pt.named_parameters():
    if name in unfreeze:
        param.requires_grad = True
    else:
        param.requires_grad = False
    
# fc layer 수정
fc_in_features = resnet_pt.fc.in_features
resnet_pt.fc = nn.Linear(fc_in_features, len(classes))
resnet_pt = resnet_pt.to(device)

In [8]:
summary(resnet_pt, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_pt.parameters(), lr=0.001,
                      momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [10]:
# Training
def train(epoch, model, criterion, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    predicted_list = 0
    labels_list = 0
    Norm_count = {"correct":0,"total":0}
    Mot_count = {"correct":0,"total":0}
    AbNorm_count = {"correct":0,"total":0}
    
    for batch_idx, (inputs, labels) in enumerate(tqdm(trainloader)):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()*inputs.size(0)
        _, predicted = outputs.max(1)
        if batch_idx == 0:
            predicted_list=predicted
        else:
            predicted_list=torch.cat((predicted_list,predicted), dim=0)
            
        if batch_idx == 0:
            labels_list=labels
        else:
            labels_list=torch.cat((labels_list, labels), dim=0)
            
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        correct_compare = predicted.eq(labels)
        for i in range(3):
            if i==0:
                Norm_count['correct'] += predicted[correct_compare].tolist().count(i)
                Norm_count['total'] += predicted.tolist().count(i)
            elif i==1:
                Mot_count['correct'] += predicted[correct_compare].tolist().count(i)
                Mot_count['total'] += predicted.tolist().count(i)
            else:
                AbNorm_count['correct'] += predicted[correct_compare].tolist().count(i)
                AbNorm_count['total'] += predicted.tolist().count(i)
            
    epoch_loss = train_loss/total
    epoch_acc = correct/total*100
    f1_score = multiclass_f1_score(predicted_list, labels_list, num_classes=3, average="macro")
    Norm = Norm_count['correct']/Norm_count['total']
    Mot = Mot_count['correct']/Mot_count['total']
    AbNorm = AbNorm_count['correct']/AbNorm_count['total']
    print(f"Test | Loss: {epoch_loss: .4f} Acc: {epoch_acc: .2f}  F1-Score: {f1_score:.2f} ({correct}/{total}) Norm:{Norm:.2f} Mot:{Mot:.2f} AbNorm:{AbNorm:.2f}")
    return epoch_loss, epoch_acc, f1_score

def test(epoch, model, criterion, optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    predicted_list = 0
    labels_list = 0
    Norm_count = {"correct":0,"total":0}
    Mot_count = {"correct":0,"total":0}
    AbNorm_count = {"correct":0,"total":0}
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(tqdm(testloader)):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()*inputs.size(0)
            _, predicted = outputs.max(1)
            if batch_idx == 0:
                predicted_list=predicted
            else:
                predicted_list=torch.cat((predicted_list,predicted), dim=0)
            
            if batch_idx == 0:
                labels_list=labels
            else:
                labels_list=torch.cat((labels_list, labels), dim=0)
                
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            correct_compare = predicted.eq(labels)
            
            for i in range(3):
                if i==0:
                    Norm_count['correct'] += predicted[correct_compare].tolist().count(i)
                    Norm_count['total'] += predicted.tolist().count(i)
                elif i==1:
                    Mot_count['correct'] += predicted[correct_compare].tolist().count(i)
                    Mot_count['total'] += predicted.tolist().count(i)
                else:
                    AbNorm_count['correct'] += predicted[correct_compare].tolist().count(i)
                    AbNorm_count['total'] += predicted.tolist().count(i)

            
        epoch_loss = test_loss/total
        epoch_acc = correct/total*100
        f1_score = multiclass_f1_score(predicted_list, labels_list, num_classes=3, average="macro")
        Norm = Norm_count['correct']/Norm_count['total']
        Mot = Mot_count['correct']/Mot_count['total']
        AbNorm = AbNorm_count['correct']/AbNorm_count['total']
        print(f"Test | Loss: {epoch_loss: .4f} Acc: {epoch_acc: .2f}  F1-Score: {f1_score:.2f} ({correct}/{total}) Norm:{Norm:.2f} Mot:{Mot:.2f} AbNorm:{AbNorm:.2f}")
    return epoch_loss, epoch_acc, f1_score

In [11]:
tart_time = time.time()
best_acc = 0
epoch_length = 20
save_loss = {"train":[],
             "test":[]}
save_acc = {"train":[],
             "test":[]}
save_f1 = {"train":[],
            "test":[]}
start_time= time.time()

for epoch in range(epoch_length):
    print("Epoch %s" % epoch)
    train_loss, train_acc, train_f1 = train(epoch, resnet_pt, criterion, optimizer)
    save_loss['train'].append(train_loss)
    save_acc['train'].append(train_acc)
    save_f1['train'].append(train_f1)
    
    test_loss, test_acc, test_f1 = test(epoch, resnet_pt, criterion, optimizer)
    save_loss['test'].append(test_loss)
    save_acc['test'].append(test_acc)
    save_f1['test'].append(test_f1)

    scheduler.step()

    # Save model
    if test_acc > best_acc:
        best_acc = test_acc
        best_model_wts = copy.deepcopy(resnet_pt.state_dict())
    resnet_pt.load_state_dict(best_model_wts)

learning_time = time.time() - start_time
print(f'**Learning time: {learning_time // 60:.0f}m {learning_time % 60:.0f}s')

Epoch 0


100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [12:56<00:00,  9.35s/it]


Test | Loss:  0.3830 Acc:  86.55  F1-Score: 0.78 (36720/42426) Norm:0.85 Mot:0.94 AbNorm:0.90


100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [01:15<00:00,  3.58s/it]


Test | Loss:  0.1659 Acc:  95.53  F1-Score: 0.94 (5069/5306) Norm:0.96 Mot:0.98 AbNorm:0.93
Epoch 1


...

Test | Loss:  0.0300 Acc:  99.02  F1-Score: 0.99 (5254/5306) Norm:0.99 Mot:0.99 AbNorm:0.98
Epoch 19


100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [13:25<00:00,  9.71s/it]


Test | Loss:  0.0180 Acc:  99.53  F1-Score: 0.99 (42225/42426) Norm:1.00 Mot:1.00 AbNorm:0.99


100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [01:18<00:00,  3.74s/it]

Test | Loss:  0.0297 Acc:  99.02  F1-Score: 0.99 (5254/5306) Norm:0.99 Mot:0.99 AbNorm:0.98
**Learning time: 282m 7s
